In [3]:
import tkinter as tk
from tkinter import messagebox
from tkinter import *
import cv2
import os
from PIL import Image
from PIL import *
import numpy as np
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pickle
import joblib

In [4]:
window = tk.Tk()

window.title("Hệ thống nhận dạng khuôn mặt")

l1 = tk.Label(window,text="MSSV", font=("Roboto", 20))
l1.grid(column=0, row=0)
t1 = tk.Entry(window, width=50, bd=5)
t1.grid(column=1, row=0)

l2 = tk.Label(window,text="Tuổi", font=("Roboto", 20))
l2.grid(column=0, row=1)
t2 = tk.Entry(window, width=50, bd=5)
t2.grid(column=1, row=1)

l3 = tk.Label(window,text="Họ và tên", font=("Roboto", 20))
l3.grid(column=0, row=2)
t3 = tk.Entry(window, width=50, bd=5)
t3.grid(column=1, row=2)

l4 = tk.Label(window,text="Ngày sinh", font=("Roboto", 20))
l4.grid(column=0, row=3)
t4 = tk.Entry(window, width=50, bd=5)
t4.grid(column=1, row=3)


def generate_dataset() :
    if t1.get() == "" or t2.get() == "" or t3.get() == "" or t4.get() == "":
        messagebox.showinfo("Result", "Hãy nhập đầy đủ các trường")
    else:
        face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        def face_cropped(img):
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray, 1.3, 5)
            if faces is () :
                return None
            for(x, y, w, h) in faces:
                cropped_face = img[y:y+h, x:x+w]
            return cropped_face
        cap = cv2.VideoCapture(0)
        id = int(t1.get())
        img_id = 0

        while True :
            ret, frame = cap.read()
            if face_cropped(frame) is not None:
                img_id += 1
                face = cv2.resize(face_cropped(frame), (200, 200))
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                file_name_path = "data/user." + str(id) + "-" + t3.get() + "." + str(img_id) + ".jpg"
                cv2.imwrite(file_name_path, face)
                cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

                cv2.imshow("Cropped face", face)
                if cv2.waitKey(1) == 13 or int(img_id) == 100:
                    break
        cap.release()
        cv2.destroyAllWindows()

b1 = tk.Button(window, text="Thêm dataset", font=("Roboto", 20), bg="orange", fg="black", command=generate_dataset)
b1.grid(column=0, row=4)


def train_classifier() :
    data_dir = "C:/Users/MINH/Downloads/face-recognition-PCA/face-recognition-PCA/data" 
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []
    
    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        text = os.path.split(image)[1].split(".")[1]
        id = int(text.split("-")[0])
        faces.append(imageNp)
        ids.append(id)
    ids = np.array(ids)
    #feature extraction
    pca = PCA(n_components=1)
    image_pca = []
    for img_vector in faces:
        pca.fit(img_vector)
        img_pca = pca.transform(img_vector)
        image_pca.append(img_pca.flatten())
    #train classifier and save it
    clf = SVC(kernel = 'rbf', class_weight='balanced', probability=True)
    clf.fit(image_pca, ids)
    joblib.dump(clf, 'classifier.pkl')
    messagebox.showinfo("Result", "Training hoàn thành!")

b2 = tk.Button(window, text="Training", font=("Roboto", 20), bg="green", fg="black", command=train_classifier)
b2.grid(column=1, row=4)

def detect_face():
    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf, pca) :
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbors)

        coords = []
        for(x, y, w, h) in features:
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            img_cropped = cv2.resize(gray_image[y:y+h,x:x+w], (200, 200))
            pca.fit(img_cropped)
            img_pca = pca.transform(img_cropped).flatten()
            pred = clf.predict([img_pca])
            proba = clf.predict_proba([img_pca])
            max_proba = max(proba.flatten())
            print(max_proba)
            if int(max_proba * 100) >= 95:
                cv2.putText(img, "MSSV: " + str(pred), (x, y-5), cv2.FONT_HERSHEY_COMPLEX, 0.8, color, 1, cv2.LINE_AA)
            else :
                cv2.putText(img, "UNKNOWN", (x, y-5), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)

            coords = [x, y, w, h]
        return coords

    def recognize(img, clf, faceScascade, pca) :
        coords = draw_boundary(img, faceCascade, 1.3, 5, (255, 255, 255), "Face", clf, pca)
        return img

    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    clf = joblib.load("classifier.pkl")
    pca = PCA(n_components=1)
    video_capture = cv2.VideoCapture(0)

    while True:
        ret, img = video_capture.read()
        img = recognize(img, clf, faceCascade, pca)
        cv2.imshow("face detection", img)

        if cv2.waitKey(1) == 13 :
            break
    video_capture.release()
    cv2.destroyAllWindows()

b3 = tk.Button(window, text="Nhận dạng", font=("Roboto", 20), bg="pink", fg="black", command=detect_face)
b3.grid(column=2, row=4)


window.geometry("1000x640")
window.mainloop()

<>:34: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:34: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\MINH\AppData\Local\Temp\ipykernel_30612\1730204121.py:34: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is () :


0.9986864097200893
0.9986864097200893
0.9979338079661406
0.9984656021608723
0.9973993628858754
0.9973993628858754
0.9946671985990503
0.9946671985990503
0.9934715482412585
0.9780029203929754
0.9780029203929754
0.9737619320845516
0.9277674122296968
0.9120387277333084
0.9120387277333084
0.7746750321177048
0.9364230417645074
0.9364230417645074
0.9367071823755897
0.5785156579148522
0.9930473470620192
0.9930473470620192
0.6858772480400532
0.7827578827605971
0.6923519853937032
0.6923519853937032
0.8616483488225167
0.6191830625260922
0.6191830625260922
0.6243782698135517
0.800128685096918
0.816581612435795
0.816581612435795
0.7875543723682934
0.8567239046270686
0.8567239046270686
0.9747076797479152
0.9886562994998895
0.9886562994998895
0.9750864921525024
0.5465245423939447
0.5465245423939447
0.8766289353783706
0.6709919130030151
0.6118241721150273
0.6118241721150273
0.7881731350868866
0.7253099792150022
0.7253099792150022
0.8463682722423344
0.6529802689089196
0.8749254410087492
0.8749254410087